Data/Image processing

In [5]:
import cv2 #cv2 stands for opencv : opensource computer vision and ml library
import numpy as np
import os #interacting with operating system- to do things like we use command line for accessing and manipulating files
import csv

# Path to the folder containing your images
folder_path = "E7-images"

output_folder = "E7-tfimages"
os.makedirs(output_folder, exist_ok=True)

# Initialize lists to store results
images =[]
tight_fitting_boxes = []
contour_areas = []
tight_fit_box_areas = []
dim_1=[]
dim_2=[]

processed_images=0

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (JPG OR JPEG or PNG)
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"): #so that we don't run into a situation where we are doing imge kund operations on some non-image files
        # Construct the full path to the image file
        img_path = os.path.join(folder_path, filename)
        
        # Load image
        image = cv2.imread(img_path) #reading specific image file
        images.append(image)
        original_image = image.copy()  # Save a copy for visualization
        
        # Convert image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #conversion flag- cv2.COLOR_BGR2GRAY
        
        # Threshold the image to create a mask of colored areas 
        #threshhold fn gives threshold value and thresholded image as output.
        _, masked_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY_INV) 
        # 200 is our threshold value. pixels above 200 get conv into 255(W) and rest into 0(B): then through binary inversion fn they are inverted -- swaped.

        # Find contours 
        #findContours function gives contours and their heirarchies
        contours, _ = cv2.findContours(masked_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
        #cv2.RETR_EXTERNAL ignores any internal contours that exist. 
        #cv2.CHAIN_APPROX_SIMPLE is contour approximation method- compresses horizontal, vertical, and diagonal segments and leaves only their end points.
        
        for contour in contours:
            # Minimum area rectangle
            rect = cv2.minAreaRect(contour) #tight-fitting box
            
            # Box coordinates
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            tight_fitting_boxes.append(box)
            
            # Calculate dimensions
            dim1 = np.linalg.norm(box[0] - box[1])
            dim2 = np.linalg.norm(box[1] - box[2])
            
            dim_1.append(dim1)
            dim_2.append(dim2)
            # Calculate contour area
            contour_area = cv2.contourArea(contour)
            contour_areas.append(contour_area)
            
            # Calculate layout area
            tight_fit_box_area = dim1 * dim2
            tight_fit_box_areas.append(tight_fit_box_area)
            
            # Draw tight-fitting box and contours on the image (for visualization)
            cv2.drawContours(original_image, [box], 0, (0, 255, 0), 2)  # Tight-fitting box
            cv2.drawContours(original_image, [contour], 0, (255, 0, 0), 2)  # Contour
        
        # Write processed image to output folder
        output_img_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_img_path, original_image)
        ##### sample print block #####
        # # Show the image with tight-fitting boxes and contours (for visualization)
        # cv2.imshow("Image with Tight-Fitting Boxes and Contours", original_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        
        # Increment the counter
        processed_images += 1
        
        # Stop after processing 5 images ##### sample print block #####
        # if processed_images == 5:
        #     break
csv_file = "tf.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Tight-Fitting Boxes (Coordinates)", "Dimension 1", "Dimension 2", "Contour Area", "Tight fit box Area"])
    for i in range(len(tight_fitting_boxes)):
        writer.writerow([tight_fitting_boxes[i], dim_1[i], dim_2[i], contour_areas[i], tight_fit_box_areas[i]])

print("Results saved to:", csv_file)

# # Print 5 samples of results  ##### sample print block #####
# for i in range(5):
#     print("Sample", i+1, "Tight-Fitting Boxes (Coordinates):", tight_fitting_boxes[i])
#     print("Sample", i+1, "Dimension 1:", dim_1[i])
#     print("Sample", i+1, "Dimension 2:", dim_2[i])
#     print("Sample", i+1, "Contour Area:", contour_areas[i])
#     print("Sample", i+1, "Tight fit box Area:", tight_fit_box_areas[i])

C:\Users\bpadm\AppData\Local\Temp\ipykernel_37340\2079432233.py:52: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


Results saved to: tf.csv


Grouping based on shapes -- autoencoders and k-means clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Flatten images
flattened_images = np.array([image.flatten() for image in images])

# Normalize pixel values to [0, 1]
flattened_images = flattened_images / 255.0

# Define and train the autoencoder
autoencoder = Sequential([
    Dense(128, activation='relu', input_shape=(flattened_images.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(flattened_images.shape[1], activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(flattened_images, flattened_images, epochs=10, batch_size=32)

# Extract features using the encoder part
encoder = Sequential(autoencoder.layers[:3])
autoencoder_features = encoder.predict(flattened_images)

# Perform KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(autoencoder_features)
labels = kmeans.labels_

# Evaluate clustering
silhouette_avg_autoencoder = silhouette_score(autoencoder_features, labels)
print("Silhouette Score (Autoencoder):", silhouette_avg_autoencoder)

# Group samples by cluster
cluster_samples_autoencoder = [[] for _ in range(5)]
for idx, label in enumerate(labels):
    cluster_samples_autoencoder[label].append(images[idx])

# Print 5 randomly picked samples from each cluster
for cluster_idx, samples in enumerate(cluster_samples_autoencoder):
    print("Cluster", cluster_idx + 1)
    random_samples = random.sample(samples, min(5, len(samples)))
    for i, sample in enumerate(random_samples):
        cv2.imshow(f"Cluster {cluster_idx + 1} Sample {i + 1}", sample)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
